<a href="https://colab.research.google.com/github/fabiomatricardi/Abstractive-Extractive/blob/main/MEDIUM_Optimized_Summarization_with_FlanT5LaMini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://uploads-ssl.webflow.com/5fdc17d51dc102ed1cf87c05/628cf8c014925ccdbc358d30_dengqingxiong_0-1585676096532.jpeg" height=300>


# Optimize Summarizations with with_FlanT5LaMini


---



---



Using Page Plain Text Chrome addon to download plain text from the Medium article
Load it on Google Colab
Use FlanT5-LaMini for Summarization and chat?

# FINAL NOTEBOOK

<img src="https://github.com/fabiomatricardi/Abstractive-Extractive/raw/main/logo.png">

# 🏡 🦙 Test with locally downloaded model

refer for text to https://towardsdatascience.com/text-summarization-with-nlp-textrank-vs-seq2seq-vs-bart-474943efeb09


In [1]:
%%capture
!pip install mkl mkl-include
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
!pip install git+https://github.com//huggingface/transformers
!pip install accelerate
!pip install rich

In [2]:
%%capture
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

In [3]:
#@title Restart Runtime {display-mode: "form"}
import ipywidgets as widgets
def restart(b):
  exit()

button2 = widgets.Button(
    description='Restart Runtime',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button2.on_click(restart)
button2


Button(button_style='warning', description='Restart Runtime', icon='check', style=ButtonStyle(), tooltip='Clic…

### Download text files

In [1]:
%%capture
!wget https://github.com/fabiomatricardi/Abstractive-Extractive/raw/main/BERTexplanation.txt
!wget https://github.com/fabiomatricardi/Abstractive-Extractive/raw/main/Text%20Summarization%20with%20NLP-%20TextRank%20vs%20Seq2Seq%20vs%20BART.txt
!wget https://github.com/fabiomatricardi/Abstractive-Extractive/raw/main/AutomaticTextSummarization.txt
!wget https://github.com/fabiomatricardi/Abstractive-Extractive/raw/main/GPT4VsLima.txt
!wget https://github.com/fabiomatricardi/Abstractive-Extractive/raw/main/nlp-basics-abstractive-and-extractive-text-summarization.txt

### Download Model from HuggingFace *MBZUAI/LaMini-Flan-T5-248M* and move it to Model folder

In [2]:
%%capture
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/.gitattributes
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/.gitignore
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/README.md
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/config.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/generation_config.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/pytorch_model.bin
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/special_tokens_map.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/spiece.model
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/tokenizer.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/tokenizer_config.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/training_args.bin
!mkdir model
!mv /content/.gitattributes /content/model/.gitattributes
!mv /content/.gitignore /content/model/.gitignore
!mv /content/README.md /content/model/README.md
!mv /content/config.json  /content/model/config.json
!mv /content/generation_config.json  /content/model/generation_config.json
!mv /content/pytorch_model.bin  /content/model/pytorch_model.bin
!mv /content/special_tokens_map.json  /content/model/special_tokens_map.json
!mv /content/spiece.model  /content/model/spiece.model
!mv /content/tokenizer.json  /content/model/tokenizer.json
!mv /content/tokenizer_config.json  /content/model/tokenizer_config.json
!mv /content/training_args.bin  /content/model/training_args.bin

### Initialize calls for Local T5 LaMini Model

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel, T5Tokenizer, T5Model
from transformers import T5ForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline
import torch
import rich
from rich.console import Console
from rich.panel import Panel
from rich import print
import ssl
import requests
import datetime
import os
import requests
import datetime
#from tqdm import tqdm
from tqdm.rich import trange, tqdm

########### SSL FOR PROXY ##############
ssl._create_default_https_context = ssl._create_unverified_context

########## INITIALIZE RICH CONSOLE  ##################
console = Console()


########## LANGUAGE MODELS CHECKPOINTS ###########################
checkpoint = "./content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine

## ⭐ BEST PIPELINE settings chunks/overlap
## ⭐ Langchain/pipelines chunk 600/100 good below 5000 character
### ⭐ above 5000 characther 1700/300 or even 2000/300

In [8]:
#################### Import TEXT file as String ###################
fname = '/content/GPT4VsLima.txt'
with open(fname) as f:
    doc = f.read()
f.close()
#################### Assign CheckPoint path for Model ###################
LaMini = '/content/model/'

#### a small text from the GPT4VsLima...

In [5]:
doc2 = """Title: GPT-4 vs LIMA: Rethinking Large Language Models for Efficiency and Performance | by Amir Shakiba | May, 2023 | Medium
GPT-4 vs LIMA: Rethinking Large Language Models for Efficiency and Performance
written by Amir Shakiba
A recent paper by Meta AI has the potential to revolutionize our understanding of large language models (LLMs).
To delve into their workings, let’s take a closer look at Meta AI’s LLAMA model. (you can jump straight to LIMA if you want)
LLAMA
LLMs, which are trained on vast amounts of text, have given us impressive results. Initially, it was believed that bigger models are necessary for better performance. However, recent papers suggest that smaller models trained on more data can actually deliver better results, challenging the notion of model size. Importantly, practical considerations come into play. In terms of production efficiency, it is more advantageous to train a smaller model for a longer duration, rather than opting for a larger model trained in a shorter time frame that requires more GPU resources during inference.
smaller models on larger datasets means less cost and more affordability leading to democratization of AI which OpenAI is really concerned about!
This is where LLAMA models come in. Despite having fewer parameters compared to GPT-3 models, LLAMA models can run on a single GPU. Additionally, LLAMA models are exclusively trained on openly accessible datasets, in contrast to other systems like ChatGPT, which rely on data that is not publicly available.(openAI or closeAI?:)
"""

In [13]:
len(doc2.split(' ')) #number of words in docs2
len(doc2)

1542

In [16]:
######################################################################
#     SUMMARIZATION FROM TEXT STRING WITH HUGGINGFACE PIPELINE       #
######################################################################
def AI_SummaryPL(checkpoint, text, chunks, overlap):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel, T5Tokenizer, T5Model
    from transformers import T5ForConditionalGeneration, pipeline
    from langchain.llms import HuggingFacePipeline
    import torch

    """
    checkpoint is in the format of relative path
    example:  checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
    text it is either a long string or a input long string or a loaded document into string
    chunks: integer, lenght of the chunks splitting
    ovelap: integer, overlap for cor attention and focus retreival

    """
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = chunks,
        chunk_overlap  = overlap,
        length_function = len,
    )
    texts = text_splitter.split_text(text)
    #checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
    checkpoint = checkpoint
    tokenizer = T5Tokenizer.from_pretrained(checkpoint)
    base_model = T5ForConditionalGeneration.from_pretrained(checkpoint,
                                                        device_map='auto',
                                                        torch_dtype=torch.float32)
    console.print("[yellow bold] Inizializing pipeline")
    pipe_sum = pipeline('summarization',
                        model = base_model,
                        tokenizer = tokenizer,
                        max_length = 650, #origina  350
                        min_length = 25
                        )

    start = datetime.datetime.now() #not used now but useful
    # taking from ipyWidget textarea content
    full_summary = ''
    for cnk in trange(len(texts)):
      result = pipe_sum(texts[cnk])
      full_summary = full_summary + ' '+ result[0]['summary_text']
    stop = datetime.datetime.now() #not used now but useful
    delta = stop-start

    print(Panel(full_summary, title='AI Summarization'))
    console.print(f"[red bold]Summarization completed in {delta}")
    console.print(f"Entire Summarization lenght: [bold bright_magenta]{len(full_summary)}[/bold bright_magenta]/{len(text)} character")
    reduction = '{:.1%}'.format(len(full_summary)/len(text))
    console.print(f"Summarization reduction: [bold bright_magenta]{reduction}")
    return full_summary


## EXTRACTIVE EXAMPLE 1

In [17]:
post_summary14 = AI_SummaryPL(LaMini,doc2,1700,20)  #3700/500 for texts > 6000 charachters

 Inizializing pipeline

Output()

/usr/local/lib/python3.10/dist-packages/tqdm/rich.py:145: TqdmExperimentalWarning: rich is experimental/alpha
  return tqdm_rich(range(*args), **kwargs)
Your max_length is set to 650, but your input_length is only 358. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=179)


╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│  Meta AI's LLAMA model, which is trained on vast amounts of text, has shown impressive results. However, recent │
│ papers suggest that smaller models trained on more data can deliver better results, challenging the notion of   │
│ model size. Smaller models can run on a single GPU and are exclusively trained on openly accessible datasets,   │
│ leading to democratization of AI. OpenAI is concerned about this.                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:23.948627

Entire Summarization lenght: 397/1542 character

Summarization reduction: 25.7%

In [18]:
print(post_summary14)

Meta AI's LLAMA model, which is trained on vast amounts of text, has shown impressive results. However, recent 
papers suggest that smaller models trained on more data can deliver better results, challenging the notion of model
size. Smaller models can run on a single GPU and are exclusively trained on openly accessible datasets, leading to 
democratization of AI. OpenAI is concerned about this.

## EXTRACTIVE EXAMPLE 2

In [19]:
doc3 ="""
BERT: A Beginner-Friendly Explanation written By Pushpam Punjabi
Up until now, we’ve seen how a computer understands the meaning of different words using word embeddings. In the last blog, we also looked at how we can take average of the embeddings of words appearing in a sentence to represent that sentence as an embedding. This is one of the ways of interpreting a sentence. But that’s not how humans understand the language. We don’t just take individual meaning of words and form the understanding of a sentence or a paragraph. A much more complex process is involved to understand language by humans. But how does a machine understand language? It’s through language models!
Language models are an essential component of Natural Language Processing (NLP), designed to understand and generate human language. They use various statistical and machine learning techniques to analyze and learn from large amounts of text data, enabling them to identify patterns and relationships between words, phrases, and sentences. Word embeddings form the base in understanding these sentences! Language models have revolutionized the field of NLP and have played a crucial role in enabling machines to interact with humans in a more natural and intuitive way. Language models have also surpassed humans in some of the tasks in NLP!
In this blog, we will understand Bi-directional Encoder Representations from Transformers (BERT) which is one of the biggest milestones in the world on language models!
Understanding BERT: BERT was developed by Google in 2018. It is a “Language Understanding” model, that is trained on a massive amounts of text data to understand the context and meaning of words and phrases in a sentence. BERT uses “transformer” deep learning architecture that enables it to process information bidirectionally, meaning it can understand the context of a word based on both, the words that come before and after it. This allows BERT to better understand the nuances of language, including idioms, sarcasm, and complex sentence structures.
You must be wondering how do you train such models to understand human language? There are 2 training steps involved to use BERT:
Pre-training phase
Fine-tuning phase
1. Pre-training phase
In pre-training phase, the model is trained on huge textual data. This is the stage where the model learns and understand the language. Pre-training is expensive. To pre-train a BERT model, Google used multiple TPUs — special computing processors for deep learning models. It took them 4 days to pre-train BERT on such a large infrastructure. But this is only a one-time procedure. Once the model understands the language, we can reuse the model for variety of tasks in NLP. There are 3 steps to pre-train BERT:
Text corpus selection
Masked Language Modeling
Next Sentence Prediction
Let’s go through each step in detail.
"""

In [20]:
len(doc3)

2860

In [21]:
post_summary15 = AI_SummaryPL(LaMini,doc3,2000,200)  #3700/500 for texts > 6000 charachters

 Inizializing pipeline

Output()

/usr/local/lib/python3.10/dist-packages/tqdm/rich.py:145: TqdmExperimentalWarning: rich is experimental/alpha
  return tqdm_rich(range(*args), **kwargs)
Your max_length is set to 650, but your input_length is only 319. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=159)
Your max_length is set to 650, but your input_length is only 359. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=179)


╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│  BERT: A Beginner-Friendly Explanation by Pushpam Punjabi explains how a machine understands language through   │
│ language models, which are essential components of Natural Language Processing (NLP) designed to understand and │
│ generate human language. They use statistical and machine learning techniques to analyze and learn from large   │
│ amounts of text data, enabling them to identify patterns and relationships between words, phrases, and          │
│ sentences. Language models have revolutionized NLP and have surpassed humans in some of the tasks in NLP. The   │
│ blog discusses the Bi-directional Encoder Representations from Transformers (BERT), a language model developed  │
│ by Google in 2018. The model is trained on massive amounts of text data to understand the context and meaning   │
│ of words and phrases in a sentence. BERT uses transformer deep learning architecture to process information     │
│ bidirectionally, allowing it to better understand the nuances of language, including idioms, sarcasm, and       │
│ complex sentence structures. Pre-training phase involves training the model on huge textual data, which is the  │
│ stage where the model learns and understands the language. Google used multiple TPUs to pre-train a BERT model, │
│ which took 4 days to train on such a large infrastructure. After training, the model can be reused for various  │
│ tasks in NLP. There are three steps involved in pre-training BERT: Text corpus selection, Masked Language       │
│ Modeling, Next Sentence Prediction.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:01:28.313227

Entire Summarization lenght: 1455/2860 character

Summarization reduction: 50.9%

In [22]:
print(post_summary15)

BERT: A Beginner-Friendly Explanation by Pushpam Punjabi explains how a machine understands language through 
language models, which are essential components of Natural Language Processing (NLP) designed to understand and 
generate human language. They use statistical and machine learning techniques to analyze and learn from large 
amounts of text data, enabling them to identify patterns and relationships between words, phrases, and sentences. 
Language models have revolutionized NLP and have surpassed humans in some of the tasks in NLP. The blog discusses 
the Bi-directional Encoder Representations from Transformers (BERT), a language model developed by Google in 2018. 
The model is trained on massive amounts of text data to understand the context and meaning of words and phrases in 
a sentence. BERT uses transformer deep learning architecture to process information bidirectionally, allowing it to
better understand the nuances of language, including idioms, sarcasm, and complex sentence structures. Pre-training
phase involves training the model on huge textual data, which is the stage where the model learns and understands 
the language. Google used multiple TPUs to pre-train a BERT model, which took 4 days to train on such a large 
infrastructure. After training, the model can be reused for various tasks in NLP. There are three steps involved in
pre-training BERT: Text corpus selection, Masked Language Modeling, Next Sentence Prediction.

## ⭐⭐⭐ BEST LangChain Summarization (text > 6000)

In [23]:
############################################################
#     SUMMARIZE TEXT WITH LANGCHAIN AND LAMINI LOCAL       #
############################################################
def AI_SummaryLangChain(checkpoint, text, chunks, overlap,method):
  if method in ['stuff', 'map_reduce', 'refine']:
    from langchain.llms import HuggingFacePipeline
    from langchain.chains.summarize import load_summarize_chain
    from langchain.chains.mapreduce import MapReduceChain
    from langchain.prompts import PromptTemplate
    from langchain.docstore.document import Document
    """
    checkpoint is in the format of relative path
    example:  checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
    text it is either a long string or a input long string or a loaded document into string
    chunks: integer, lenght of the chunks splitting
    ovelap: integer, overlap for cor attention and focus retreival
    Got unsupported chain type: map-reduce. Should be one of dict_keys(['stuff', 'map_reduce', 'refine']
    """
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.document_loaders import TextLoader
    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = chunks,
        chunk_overlap  = overlap,
        length_function = len,
    )
    #IMPORTANT## Here we need a Document object from langchain.docstore.document
    #Because the Chain requires PAGES elements
    docs = text_splitter.create_documents([text])
    #docs = [Document(page_content=t) for t in texts] #old method

    #checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
    checkpoint = checkpoint
    tokenizer = T5Tokenizer.from_pretrained(checkpoint)
    base_model = T5ForConditionalGeneration.from_pretrained(checkpoint,
                                                        device_map='auto',
                                                        torch_dtype=torch.float32)
    console.print("[yellow bold] Inizializing pipeline")
    ##############################################################################
    #To use, you should have the transformers python package installed.
    #Only supports text-generation, text2text-generation and summarization for now.
    llm = HuggingFacePipeline.from_model_id(model_id=checkpoint,
                                            task = 'text2text-generation',
                                            model_kwargs={"temperature":0, "max_length":1000})

    ###CHAIN TYPE FOR SUMMARIZATION Should be one of dict_keys(['stuff', 'map_reduce', 'refine']
    chain = load_summarize_chain(llm, chain_type=method)
    console.print(f"[deep_pink4 bold] Entering Sumarization chain [italic]{method}[/italic] chain type")
    start = datetime.datetime.now() #not used now but useful
    # taking from ipyWidget textarea content
    full_summary = chain.run(docs)
    stop = datetime.datetime.now() #not used now but useful
    delta = stop-start
    print(Panel(full_summary, title='AI Summarization'))
    console.print(f"[red bold]Summarization completed in {delta}")
    console.print(f"Entire Summarization lenght: [bold bright_magenta]{len(full_summary)}[/bold bright_magenta]/{len(text)} character")
    reduction = '{:.1%}'.format(len(full_summary)/len(text))
    console.print(f"Summarization reduction: [bold bright_magenta]{reduction}")
    return full_summary
  else:
    pass

### Abstractive Summarization 1

In [27]:
abstractive_summary01 = AI_SummaryLangChain(LaMini,doc3,1500,300,'refine')  #refine

 Inizializing pipeline

 Entering Sumarization chain refine chain type

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The blog discusses how language models are essential in NLP and how they can be used to understand human        │
│ language. It explains how BERT is a language understanding model developed by Google in 2018. BERT is trained   │
│ on massive amounts of text data to understand the context and meaning of words and phrases in a sentence. It    │
│ uses transformer deep learning architecture to process information bidirectionally, allowing it to understand   │
│ the context of a word based on both the words that come before and after it. Pre-training is expensive and      │
│ Google used multiple TPUs to train a BERT model. Once the model understands the language, we can reuse it for   │
│ various tasks in NLP. The blog provides more context to refine the existing summary.                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:58.899842

Entire Summarization lenght: 735/2860 character

Summarization reduction: 25.7%

In [28]:
print(abstractive_summary01)

The blog discusses how language models are essential in NLP and how they can be used to understand human language. 
It explains how BERT is a language understanding model developed by Google in 2018. BERT is trained on massive 
amounts of text data to understand the context and meaning of words and phrases in a sentence. It uses transformer 
deep learning architecture to process information bidirectionally, allowing it to understand the context of a word 
based on both the words that come before and after it. Pre-training is expensive and Google used multiple TPUs to 
train a BERT model. Once the model understands the language, we can reuse it for various tasks in NLP. The blog 
provides more context to refine the existing summary.

### Abstractive Summarization 2

In [29]:
abstractive_summary02 = AI_SummaryLangChain(LaMini,doc3,1500,300,'map_reduce')  #map_reduce

 Inizializing pipeline

 Entering Sumarization chain map_reduce chain type

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The blog discusses the importance of language models in NLP and how they can be used to understand human        │
│ language. The BERT model is a language understanding model that is trained on text data to understand the       │
│ context and meaning of words and phrases in a sentence. It uses transformer deep learning architecture to       │
│ process information bidirectionally and can be reused for various tasks in NLP. The article also discusses the  │
│ process of selecting text corpus and predicting the next sentence prediction for a simulated language model.    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:52.058032

Entire Summarization lenght: 536/2860 character

Summarization reduction: 18.7%

In [30]:
print(abstractive_summary02)

The blog discusses the importance of language models in NLP and how they can be used to understand human language. 
The BERT model is a language understanding model that is trained on text data to understand the context and meaning
of words and phrases in a sentence. It uses transformer deep learning architecture to process information 
bidirectionally and can be reused for various tasks in NLP. The article also discusses the process of selecting 
text corpus and predicting the next sentence prediction for a simulated language model.

### Abstractive Summarization 3

In [31]:
abstractive_summary03 = AI_SummaryLangChain(LaMini,doc3,1500,300,'stuff')  #stuff

 Inizializing pipeline

 Entering Sumarization chain stuff chain type

Token indices sequence length is longer than the specified maximum sequence length for this model (706 > 512). Running this sequence through the model will result in indexing errors


╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The blog discusses how language models are used in Natural Language Processing (NLP) to understand and generate │
│ human language. The blog also discusses Bi-directional Encoder Representations from Transformers (BERT), which  │
│ is one of the biggest milestones in the field. The blog also discusses how to train BERT, a language            │
│ understanding model developed by Google in 2018, using a large amount of textual data. The model is trained on  │
│ a multifaceted architecture that allows it to process information bidirectionally, allowing it to better        │
│ understand the nuances of language. The blog also mentions how to reuse the model for various tasks in NLP.     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:26.499152

Entire Summarization lenght: 647/2860 character

Summarization reduction: 22.6%

In [32]:
print(abstractive_summary03)

The blog discusses how language models are used in Natural Language Processing (NLP) to understand and generate 
human language. The blog also discusses Bi-directional Encoder Representations from Transformers (BERT), which is 
one of the biggest milestones in the field. The blog also discusses how to train BERT, a language understanding 
model developed by Google in 2018, using a large amount of textual data. The model is trained on a multifaceted 
architecture that allows it to process information bidirectionally, allowing it to better understand the nuances of
language. The blog also mentions how to reuse the model for various tasks in NLP.

========//==============//==============//=============

### Instructions
- Load the file into a `doc` variable
- run AI_SummaryPL (3700,500) for long Summary
- run AI_SummaryLangChain (1500,300,'refine') for shorter one

In [ ]:
#################### Import TEXT file as String ###################
fname = '/content/nlp-basics-abstractive-and-extractive-text-summarization.txt'
with open(fname) as f:
    doc3 = f.read()
f.close()